In [1]:
import numpy as np

## Loading the training set
I use UTKFACE Dataset including 23,708 Faces and their true ages as training set.

### Loading labels of the training set


In [2]:
y_train_PATH = "../../Datasets/UTKFace/utkface_dist.txt"
y_train = np.loadtxt(y_train_PATH, delimiter=' ')
y_train.shape

(23708, 116)

### Data Generator, Data Augmenter & Data Loader

In [3]:
import os
from PIL import Image
from torchvision import io, transforms
import torch
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

UTKFACE_PATH = "../../Datasets/UTKFace/UTKFace/UTKFace/"


class DataGenerator(Dataset):
 
    def __init__(self, path, image_size, y, transform):
        self.images_list = os.listdir(path)
        self.images_list.sort()
        self.image_size = image_size
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.images_list)

    def __getitem__(self, idx):
        img = io.read_image(UTKFACE_PATH+self.images_list[idx], mode = io.ImageReadMode.RGB).float()
        img = self.transform(img)
        label = self.y[idx]
        return img, label

image_size = (200,200)
batch_size = 32

augmentation = transforms.Compose([
        transforms.Resize(torch.Size([image_size[0], image_size[1]])),
        #transforms.RandomResizedCrop(image_size),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

train_dataset = DataGenerator(path=UTKFACE_PATH, image_size = image_size, y = y_train, transform=augmentation)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)


## Creating Test sets
I Use whole of FG-NET dataset and 10% of AgeDB to evaluate the model.

In [4]:
from torchvision.io import ImageReadMode
from scipy.special import softmax

FGNET_PATH = '../../Datasets/FGNET/FGNET/images/'
fgnet_addrs = os.listdir(FGNET_PATH)
X_val = []
Y_val=[]
y_test = []
def normal_dist(x , mu , std):
    prob_density = (np.pi*std) * np.exp(-0.5*((x-mu)/std)**2)
    return prob_density

utkface_N = 116 # number of ages in the dataset
sigma = 5

for image_name in os.listdir(FGNET_PATH):
    true_age = int(image_name[4:6])
    img = io.read_image(FGNET_PATH+image_name, mode = ImageReadMode.RGB).int()
    img = transforms.Resize(torch.Size([image_size[0], image_size[1]]))(img)
    X_val.append(np.array(img))
    mu = true_age
    generated_numbers = np.random.uniform(1, utkface_N, utkface_N)
    generated_numbers.sort()
    y = softmax(normal_dist(generated_numbers, mu, sigma))
    Y_val.append(y)
    y_test.append(true_age)

X_val, Y_val, y_test = torch.Tensor(np.asarray(X_val)), torch.Tensor(np.asarray(Y_val)), np.asarray(y_test)

from sklearn.model_selection import train_test_split
AGEDB_PATH = '../../Datasets/AgeDB/AgeDB/'
agedb_addrs = os.listdir(AGEDB_PATH)
_ , val_images_addr = train_test_split(agedb_addrs, test_size=0.1 ,random_state=49)
X_test_agedb = []
y_test_agedb = []
for addr in val_images_addr:
    img = io.read_image(AGEDB_PATH+addr, mode = ImageReadMode.RGB).int()
    img = transforms.Resize(torch.Size([image_size[0], image_size[1]]))(img)
    X_test_agedb.append(np.asarray(img))
    y_test_agedb.append(int(addr.split('_')[1][:-4]))

X_test_agedb, y_test_agedb = torch.Tensor(np.asarray(X_test_agedb)), np.asarray(y_test_agedb)

## VGG-16 Model

In [5]:
import torch.nn as nn
from torchvision.models import vgg16

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
def create_model():
  base_model = vgg16(pretrained=True)
  for param in base_model.parameters():
    param.requires_grad = False
  base_model.classifier[6] = nn.Linear(in_features=4096, out_features=116)
  base_model = base_model.to(device)
  return base_model

## Traning Loop
train(model, loss_fn, optimizer) is the main function to train the models. 

In [6]:
import gc
from torch.nn import Softmax
from torch.optim import Adam, AdamW, SGD

def train(model, loss_fn, optimizer):
    model.train()
    total_loss = 0
    softmax_func = Softmax(dim=1)
    total = len(train_dataloader)

    for i, batch in enumerate(train_dataloader):
        step = i+1
        percent = "{0:.5f}".format(100 * (step / float(total)))
        lossp = "{0:.5f}".format(total_loss/(step*batch_size))
        filledLength = int(100 * step // total)
        bar = '█' * filledLength + '>'  *(filledLength < 100) + '.' * (99 - filledLength)
        print(f'\rBatch {step}/{total} |{bar}| {percent}% complete, loss={lossp}', end='')
        batch = [r.to(device) for r in batch]
        imgs, labels = batch
        del batch
        gc.collect()
        torch.cuda.empty_cache()
        model.zero_grad()
        preds = model(imgs.float())
        preds = softmax_func(preds)
        loss = loss_fn(preds, labels)
        total_loss += float(loss.item())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    gc.collect()
    torch.cuda.empty_cache()

    avg_loss = total_loss / (len(train_dataloader)*batch_size)
    
    return avg_loss

def evaluate(model, loss_fn):
    val_loss = 0
    model.eval()
    softmax_func = Softmax(dim=1)
    with torch.no_grad():
        for i in range(0, X_val.size()[0], batch_size):
            gc.collect()
            torch.cuda.empty_cache()
            preds = model(X_val[i:i+batch_size].float().to(device))
            labels = torch.Tensor(Y_val[i:i+batch_size , :])
            labels = labels.to(device)
            preds = softmax_func(preds)
            loss = loss_fn(preds.double(), labels.double())
            val_loss += loss.item()
        return  val_loss / X_val.size()[0]   
        

## Loss functions

In [7]:
def KL(y_hat, y):
  return torch.sum(y*torch.log(y/(y_hat+1e-8)))

def GJM(y_hat, y):
  return torch.sum(y*torch.square(1-torch.sqrt((y_hat+1e-8)/(y))))

## Optimizer: Adam
In this and the next sections, I train models with related optimizers and loss functions.

### KL loss

In [71]:
vgg_adam_kl = create_model()
optimizer = Adam(vgg_adam_kl.classifier.parameters(), lr=1e-5)
loss_fn = KL
epochs = 5
current = 1
history_vgg_adam_kl = []
# for each epoch
while current <= epochs:

    print(f'\nEpoch {current} / {epochs}:')

    # train model
    train_loss = train(model=vgg_adam_kl, loss_fn = loss_fn, optimizer=optimizer)

    val_loss = evaluate(model=vgg_adam_kl, loss_fn=loss_fn)

    # evaluate model
    
    print(f'\n\nTraining Loss: {train_loss:.5f}')
    print(f'Val Loss: {val_loss:.5f}')

    history_vgg_adam_kl.append((train_loss, val_loss))

    current = current + 1
    


Epoch 1 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=15.48862

Training Loss: 15.50638
Val Loss: 6.29654

Epoch 2 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=15.15028

Training Loss: 15.16732
Val Loss: 6.31942

Epoch 3 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=15.02368

Training Loss: 15.04146
Val Loss: 6.14961

Epoch 4 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=14.97515

Training Loss: 14.99284
Val Loss: 6.11229

Epoch 5 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=14.9187

### Save The Model

In [85]:
torch.save(vgg_adam_kl.state_dict(), '../checkpoints/VGG_UTKFACE/VGG16_UTKFACE_Adam_KL.pth')

### GJM Loss

In [78]:
vgg_adam_gjm = create_model()
history_vgg_adam_gjm = []
optimizer = Adam(vgg_adam_gjm.parameters(), lr=2e-5)
loss_fn = GJM
epochs = 5
current = 1
# for each epoch
while current <= epochs:

    print(f'\nEpoch {current} / {epochs}:')

    # train model
    train_loss = train(model=vgg_adam_gjm, loss_fn = loss_fn, optimizer=optimizer)

    val_loss = evaluate(model=vgg_adam_gjm, loss_fn=loss_fn)

    # evaluate model
    
    print(f'\n\nTraining Loss: {train_loss:.5f}')
    print(f'Val Loss: {val_loss:.5f}')

    history_vgg_adam_gjm.append((train_loss, val_loss))

    current = current + 1


Epoch 1 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.83603

Training Loss: 1.83816
Val Loss: 1.83253

Epoch 2 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.78993

Training Loss: 1.79208
Val Loss: 1.81164

Epoch 3 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.77898

Training Loss: 1.78108
Val Loss: 1.77436

Epoch 4 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.77313

Training Loss: 1.77525
Val Loss: 1.75112

Epoch 5 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.77160

Traini

### Save The Model

In [84]:
torch.save(vgg_adam_gjm.state_dict(), '../checkpoints/VGG_UTKFACE/VGG16_UTKFACE_Adam_GJM.pth')

### Test on FG-NET

In [10]:
# Evalluation Metrics
def MAE(y_true, y_pred):
   M = y_pred.shape[0]
   return np.sum(np.absolute(y_true - y_pred))/M

def CS(y_true, y_pred):
   M = y_pred.shape[0]
   I = 5
   errors = np.absolute(y_true - y_pred)
   M_I = errors[errors < I].shape[0]
   return M_I/M

In [72]:
#X_test_fgnet = torch.Tensor(X_val)
vgg_adam_kl.eval()
y_pred_adam_kl = []
with torch.no_grad():
  for i in range(0, X_val.size()[0], 100):
    gc.collect()
    torch.cuda.empty_cache()
    preds = vgg_adam_kl(X_val[i:i+100].float().to(device))
    preds = preds.detach().cpu().numpy()
    pred_indices = preds.argmax(axis=1) + 1
    y_pred_adam_kl += list(pred_indices)

mae_adam_kl = MAE(y_test, np.array(y_pred_adam_kl))
cs_adam_kl = CS(y_test, np.array(y_pred_adam_kl))
print("VGG16: Training Set = UTK_FACE, optimizer = Adam, loss=KL")
print("Tested on FG-NET")
print(10*'-')
print("MAE:", mae_adam_kl)
print("CS:", cs_adam_kl) 

VGG16: Training Set = UTK_FACE, optimizer = Adam, loss=KL
Tested on FG-NET
----------
MAE: 14.410179640718562
CS: 0.156686626746507


In [79]:
#X_test_fgnet = torch.Tensor(X_val)
y_pred_adam_gjm = []
vgg_adam_gjm.eval()
with torch.no_grad():
  for i in range(0, X_val.size()[0], 100):
    gc.collect()
    torch.cuda.empty_cache()
    preds = vgg_adam_gjm(X_val[i:i+100].float().to(device))
    preds = preds.detach().cpu().numpy()
    pred_indices = preds.argmax(axis=1) + 1
    y_pred_adam_gjm += list(pred_indices)

mae_adam_gjm = MAE(y_test, np.array(y_pred_adam_gjm))
cs_adam_gjm = CS(y_test, np.array(y_pred_adam_gjm))
print("VGG16: Training Set = UTK_FACE, optimizer = Adam, loss=GJM")
print("Tested on FG-NET")
print(10*'-')
print("MAE:", mae_adam_gjm)
print("CS:", cs_adam_gjm) 

VGG16: Training Set = UTK_FACE, optimizer = Adam, loss=GJM
Tested on FG-NET
----------
MAE: 14.10878243512974
CS: 0.17065868263473055


### Test on AgeDB

In [80]:
#X_test_agedb = torch.Tensor(X_test_agedb)
y_pred_adam_kl = []
vgg_adam_kl.eval()
with torch.no_grad():
  for i in range(0, X_test_agedb.size()[0], 100):
    gc.collect()
    torch.cuda.empty_cache()
    preds = vgg_adam_kl(X_test_agedb[i:i+100].float().to(device))
    preds = preds.detach().cpu().numpy()
    pred_indices = preds.argmax(axis=1) + 1
    y_pred_adam_kl += list(pred_indices)

mae_adam_kl = MAE(y_test_agedb, np.array(y_pred_adam_kl))
cs_adam_kl = CS(y_test_agedb, np.array(y_pred_adam_kl))
print("VGG16: Training Set = UTK_FACE, optimizer = Adam, loss=KL")
print("Tested on AgeDB")
print(10*'-')
print("MAE:", mae_adam_kl)
print("CS:", cs_adam_kl) 

VGG16: Training Set = UTK_FACE, optimizer = Adam, loss=KL
Tested on AgeDB
----------
MAE: 22.76652516676774
CS: 0.16130988477865374


In [81]:
#X_test_agedb = torch.Tensor(X_test_agedb)
vgg_adam_gjm.eval()
y_pred_adam_gjm = []
with torch.no_grad():
  for i in range(0, X_test_agedb.size()[0], 100):
    gc.collect()
    torch.cuda.empty_cache()
    preds = vgg_adam_gjm(X_test_agedb[i:i+100].float().to(device))
    preds = preds.detach().cpu().numpy()
    pred_indices = preds.argmax(axis=1) + 1
    y_pred_adam_gjm += list(pred_indices)

mae_adam_gjm = MAE(y_test_agedb, np.array(y_pred_adam_gjm))
cs_adam_gjm = CS(y_test_agedb, np.array(y_pred_adam_gjm))
print("VGG16: Training Set = UTK_FACE, optimizer = Adam, loss=GJM")
print("Tested on AgeDB")
print(10*'-')
print("MAE:", mae_adam_gjm)
print("CS:", cs_adam_gjm) 

VGG16: Training Set = UTK_FACE, optimizer = Adam, loss=GJM
Tested on AgeDB
----------
MAE: 21.53426318981201
CS: 0.17040630685263797


## Optimizer: SGD

### KL loss

In [8]:
vgg_sgd_kl = create_model()
history_vgg_sgd_kl = []
optimizer = SGD(vgg_sgd_kl.parameters(), lr=0.001, momentum=0.9)
loss_fn = KL
epochs = 5
current = 1
# for each epoch
while current <= epochs:

    print(f'\nEpoch {current} / {epochs}:')

    # train model
    train_loss = train(model=vgg_sgd_kl, loss_fn = loss_fn, optimizer=optimizer)

    val_loss = evaluate(model=vgg_sgd_kl, loss_fn=loss_fn)

    # evaluate model
    
    print(f'\n\nTraining Loss: {train_loss:.5f}')
    print(f'Val Loss: {val_loss:.5f}')

    history_vgg_sgd_kl.append((train_loss, val_loss))

    current = current + 1
    


Epoch 1 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=15.43445

Training Loss: 15.45236
Val Loss: 8.09193

Epoch 2 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=14.96421

Training Loss: 14.98147
Val Loss: 7.60300

Epoch 3 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=14.79469

Training Loss: 14.81133
Val Loss: 7.84341

Epoch 4 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=14.65012

Training Loss: 14.66817
Val Loss: 7.94022

Epoch 5 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=14.4718

### Save The Model

In [13]:
torch.save(vgg_sgd_kl.state_dict(), '../checkpoints/VGG_UTKFACE/VGG16_UTKFACE_SGD_KL.pth')

### GJM loss

In [16]:
vgg_sgd_gjm = create_model()
history_vgg_sgd_gjm = []
optimizer = SGD(vgg_sgd_gjm.parameters(), lr=0.001, momentum=0.9)
loss_fn = GJM
epochs = 5
current = 1
# for each epoch
while current <= epochs:

    print(f'\nEpoch {current} / {epochs}:')

    # train model
    train_loss = train(model=vgg_sgd_gjm, loss_fn = loss_fn, optimizer=optimizer)

    val_loss = evaluate(model=vgg_sgd_gjm, loss_fn=loss_fn)
    
    # evaluate model
    
    print(f'\n\nTraining Loss: {train_loss:.5f}')
    print(f'Val Loss: {val_loss:.5f}')

    history_vgg_sgd_gjm.append((train_loss, val_loss))

    current = current + 1
    


Epoch 1 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.85086

Training Loss: 1.85313
Val Loss: 1.92149

Epoch 2 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.82209

Training Loss: 1.82416
Val Loss: 1.91487

Epoch 3 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.81681

Training Loss: 1.81886
Val Loss: 1.89890

Epoch 4 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.81527

Training Loss: 1.81733
Val Loss: 1.87917

Epoch 5 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.81842

Traini

### Save The Model

In [85]:
torch.save(vgg_sgd_gjm.state_dict(), '../checkpoints/VGG_UTKFACE/VGG16_UTKFACE_SGD_GJM.pth')

### Test on FG-NET

In [11]:
import gc
X_test_fgnet = X_val
vgg_sgd_kl.eval()
y_pred_sgd_kl = []
with torch.no_grad():
  for i in range(0, X_test_fgnet.size()[0], 100):
    gc.collect()
    torch.cuda.empty_cache()
    preds = vgg_sgd_kl(X_test_fgnet[i:i+100].float().to(device))
    preds = preds.detach().cpu().numpy()
    pred_indices = preds.argmax(axis=1) + 1
    y_pred_sgd_kl += list(pred_indices)
    
mae_sgd_kl = MAE(y_test, np.array(y_pred_sgd_kl))
cs_sgd_kl = CS(y_test, np.array(y_pred_sgd_kl))
print("VGG16: Training Set = UTK_FACE, optimizer = SGD, loss=KL")
print("Tested on FG-NET")
print(10*'-')
print("MAE:", mae_sgd_kl)
print("CS:", cs_sgd_kl) 

VGG16: Training Set = UTK_FACE, optimizer = SGD, loss=KL
Tested on FG-NET
----------
MAE: 14.349301397205588
CS: 0.1596806387225549


In [17]:
X_test_fgnet = X_val
vgg_sgd_gjm.eval()
y_pred_sgd_gjm = []
with torch.no_grad():
  for i in range(0, X_test_fgnet.size()[0], 100):
    gc.collect()
    torch.cuda.empty_cache()
    preds = vgg_sgd_gjm(X_test_fgnet[i:i+100].float().to(device))
    preds = preds.detach().cpu().numpy()
    pred_indices = preds.argmax(axis=1) + 1
    y_pred_sgd_gjm += list(pred_indices)

mae_sgd_gjm = MAE(y_test, np.array(y_pred_sgd_gjm))
cs_sgd_gjm = CS(y_test, np.array(y_pred_sgd_gjm))
print("VGG16: Training Set = UTK_FACE, optimizer = SGD, loss=GJM")
print("Tested on FG-NET")
print(10*'-')
print("MAE:", mae_sgd_gjm)
print("CS:", cs_sgd_gjm) 

VGG16: Training Set = UTK_FACE, optimizer = SGD, loss=GJM
Tested on FG-NET
----------
MAE: 13.987025948103792
CS: 0.1506986027944112


### Test on AgeDB

In [12]:
#X_test_agedb = torch.Tensor(X_test_agedb)
y_pred_sgd_kl = []
vgg_sgd_kl.eval()
with torch.no_grad():
  for i in range(0, X_test_agedb.size()[0], 100):
    gc.collect()
    torch.cuda.empty_cache()
    preds = vgg_sgd_kl(X_test_agedb[i:i+100].float().to(device))
    preds = preds.detach().cpu().numpy()
    pred_indices = preds.argmax(axis=1) + 1
    y_pred_sgd_kl += list(pred_indices)

mae_sgd_kl = MAE(y_test_agedb, np.array(y_pred_sgd_kl))
cs_sgd_kl = CS(y_test_agedb, np.array(y_pred_sgd_kl))
print("VGG16: Training Set = UTK_FACE, optimizer = SGD, loss=KL")
print("Tested on AgeDB")
print(10*'-')
print("MAE:", mae_sgd_kl)
print("CS:", cs_sgd_kl) 

VGG16: Training Set = UTK_FACE, optimizer = SGD, loss=KL
Tested on AgeDB
----------
MAE: 21.469375379017585
CS: 0.16979987871437235


In [18]:
#X_test_agedb = torch.Tensor(X_test_agedb)
vgg_sgd_gjm.eval()
y_pred_sgd_gjm = []
with torch.no_grad():
  for i in range(0, X_test_agedb.size()[0], 100):
    gc.collect()
    torch.cuda.empty_cache()
    preds = vgg_sgd_gjm(X_test_agedb[i:i+100].float().to(device))
    preds = preds.detach().cpu().numpy()
    pred_indices = preds.argmax(axis=1) + 1
    y_pred_sgd_gjm += list(pred_indices)

mae_sgd_gjm = MAE(y_test_agedb, np.array(y_pred_sgd_gjm))
cs_sgd_gjm = CS(y_test_agedb, np.array(y_pred_sgd_gjm))
print("VGG16: Training Set = UTK_FACE, optimizer = SGD, loss=GJM")
print("Tested on AgeDB")
print(10*'-')
print("MAE:", mae_sgd_gjm)
print("CS:", cs_sgd_gjm) 

VGG16: Training Set = UTK_FACE, optimizer = SGD, loss=GJM
Tested on AgeDB
----------
MAE: 21.76652516676774
CS: 0.16312916919345058


## Optimizer: AdamW

### KL loss

In [86]:
from torch.optim.lr_scheduler import MultiStepLR
vgg_adamw_kl = create_model()
history_vgg_adamw_kl = []
optimizer = AdamW(vgg_adamw_kl.parameters(), lr=1e-5, weight_decay=0.9)
scheduler = MultiStepLR(optimizer, milestones=[3], gamma=0.1)
loss_fn = KL
epochs = 5
current = 1
# for each epoch
while current <= epochs:

    print(f'\nEpoch {current} / {epochs}:')

    # train model
    train_loss = train(model=vgg_adamw_kl, loss_fn = loss_fn, optimizer=optimizer)

    val_loss = evaluate(model=vgg_adamw_kl, loss_fn=loss_fn)

    # evaluate model
    
    print(f'\n\nTraining Loss: {train_loss:.5f}')
    print(f'Val Loss: {val_loss:.5f}')

    history_vgg_adamw_kl.append(abs(train_loss - val_loss))

    scheduler.step()

    current = current + 1
    


Epoch 1 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=15.56572

Training Loss: 15.58305
Val Loss: 6.25897

Epoch 2 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=15.16692

Training Loss: 15.18428
Val Loss: 6.26711

Epoch 3 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=15.05836

Training Loss: 15.07585
Val Loss: 6.03557

Epoch 4 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=14.94489

Training Loss: 14.96197
Val Loss: 6.02457

Epoch 5 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=14.9857

### Save The Model

In [33]:
torch.save(vgg_adamw_kl.state_dict(), '../checkpoints/VGG_UTKFACE/VGG16_UTKFACE_AdamW_KL.pth')

NameError: name 'vgg_adamw_kl' is not defined

### GJM loss

In [89]:
import gc
from torch.optim.lr_scheduler import MultiStepLR
gc.collect()
torch.cuda.empty_cache()
vgg_adamw_gjm = create_model()
history_vgg_adamw_gjm = []
optimizer = AdamW(vgg_adamw_gjm.parameters(), lr=2e-5, weight_decay=0.9)
scheduler = MultiStepLR(optimizer, milestones=[3], gamma=0.1)
loss_fn = GJM
epochs = 5
current = 1
# for each epoch
while current <= epochs:

    print(f'\nEpoch {current} / {epochs}:')

    # train model
    train_loss = train(model=vgg_adamw_gjm, loss_fn = loss_fn, optimizer=optimizer)

    val_loss = evaluate(model=vgg_adamw_gjm, loss_fn=loss_fn)

    # evaluate model
    
    print(f'\n\nTraining Loss: {train_loss:.5f}')
    print(f'Val Loss: {val_loss:.5f}')

    history_vgg_adamw_gjm.append(abs(train_loss - val_loss))

    scheduler.step()

    current = current + 1


Epoch 1 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.85466

Training Loss: 1.85679
Val Loss: 1.81904

Epoch 2 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.80396

Training Loss: 1.80614
Val Loss: 1.80091

Epoch 3 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.78916

Training Loss: 1.79113
Val Loss: 1.79627

Epoch 4 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.77958

Training Loss: 1.78182
Val Loss: 1.78735

Epoch 5 / 5:
Batch 741/741 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% complete, loss=1.77606

Traini

### Save The Model

In [19]:
torch.save(vgg_adamw_gjm.state_dict(), '../checkpoints/VGG_UTKFACE/VGG16_UTKFACE_AdamW_GJM.pth')

NameError: name 'vgg_adamw_gjm' is not defined

### Test on FG-NET

In [87]:
import gc
X_test_fgnet = X_val
y_pred_adamw_kl = []
vgg_adamw_kl.eval()
with torch.no_grad():
  for i in range(0, X_test_fgnet.size()[0], 100):
    gc.collect()
    torch.cuda.empty_cache()
    preds = vgg_adamw_kl(X_test_fgnet[i:i+100].float().to(device))
    preds = preds.detach().cpu().numpy()
    pred_indices = preds.argmax(axis=1) + 1
    y_pred_adamw_kl += list(pred_indices)

mae_adamw_kl = MAE(y_test, np.array(y_pred_adamw_kl))
cs_adamw_kl = CS(y_test, np.array(y_pred_adamw_kl))
print("VGG16: Training Set = UTK_FACE, optimizer = AdamW, loss=KL")
print("Tested on FG-NET")
print(10*'-')
print("MAE:", mae_adamw_kl)
print("CS:", cs_adamw_kl) 

VGG16: Training Set = UTK_FACE, optimizer = AdamW, loss=KL
Tested on FG-NET
----------
MAE: 15.902195608782435
CS: 0.12874251497005987


In [90]:
X_test_fgnet = X_val
y_pred_adamw_gjm = []
vgg_adamw_gjm.eval()
with torch.no_grad():
  for i in range(0, X_test_fgnet.size()[0], 100):
    gc.collect()
    torch.cuda.empty_cache()
    preds = vgg_adamw_gjm(X_test_fgnet[i:i+100].float().to(device))
    preds = preds.detach().cpu().numpy()
    pred_indices = preds.argmax(axis=1) + 1
    y_pred_adamw_gjm += list(pred_indices)

mae_adamw_gjm = MAE(y_test, np.array(y_pred_adamw_gjm))
cs_adamw_gjm = CS(y_test, np.array(y_pred_adamw_gjm))
print("VGG16: Training Set = UTK_FACE, optimizer = AdamW, loss=GJM")
print("Tested on FG-NET")
print(10*'-')
print("MAE:", mae_adamw_gjm)
print("CS:", cs_adamw_gjm) 

VGG16: Training Set = UTK_FACE, optimizer = AdamW, loss=GJM
Tested on FG-NET
----------
MAE: 14.68063872255489
CS: 0.14271457085828343


### Test on AgeDB

In [88]:
#X_test_agedb = torch.Tensor(X_test_agedb)
y_pred_adamw_kl = []
vgg_adamw_kl.eval()
with torch.no_grad():
  for i in range(0, X_test_agedb.size()[0], 100):
    gc.collect()
    torch.cuda.empty_cache()
    preds = vgg_adamw_kl(X_test_agedb[i:i+100].float().to(device))
    preds = preds.detach().cpu().numpy()
    pred_indices = preds.argmax(axis=1) + 1
    y_pred_adamw_kl += list(pred_indices)

mae_adamw_kl = MAE(y_test_agedb, np.array(y_pred_adamw_kl))
cs_adamw_kl = CS(y_test_agedb, np.array(y_pred_adamw_kl))
print("VGG16: Training Set = UTK_FACE, optimizer = AdamW, loss=KL")
print("Tested on AgeDB")
print(10*'-')
print("MAE:", mae_adamw_kl)
print("CS:", cs_adamw_kl) 

VGG16: Training Set = UTK_FACE, optimizer = AdamW, loss=KL
Tested on AgeDB
----------
MAE: 22.09945421467556
CS: 0.16616130988477865


In [91]:
#X_test_agedb = torch.Tensor(X_test_agedb)
y_pred_adamw_gjm = []
vgg_adamw_gjm.eval()
with torch.no_grad():
  for i in range(0, X_test_agedb.size()[0], 100):
    gc.collect()
    torch.cuda.empty_cache()
    preds = vgg_adamw_gjm(X_test_agedb[i:i+100].float().to(device))
    preds = preds.detach().cpu().numpy()
    pred_indices = preds.argmax(axis=1) + 1
    y_pred_adamw_gjm += list(pred_indices)

mae_adamw_gjm = MAE(y_test_agedb, np.array(y_pred_adamw_gjm))
cs_adamw_gjm = CS(y_test_agedb, np.array(y_pred_adamw_gjm))
print("VGG16: Training Set = UTK_FACE, optimizer = AdamW, loss=GJM")
print("Tested on AgeDB")
print(10*'-')
print("MAE:", mae_adamw_gjm)
print("CS:", cs_adamw_gjm) 

VGG16: Training Set = UTK_FACE, optimizer = AdamW, loss=GJM
Tested on AgeDB
----------
MAE: 21.4990903577926
CS: 0.15463917525773196
